In [1]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
import numpy as np
import qiskit
%matplotlib inline
import random

In [2]:
# Step0

n=8  # length of message = n, n must be multiple of 4.

def random_01str(list,length):
    for i in range(length):
        list.append(random.randint(0,1))
        
message = []
message.clear()
random_01str(message,n)
print("message : {}".format(message))

message : [1, 1, 1, 0, 1, 0, 0, 0]


In [3]:
def XOR_message(list1,list2):
    list_part1=[]
    list_part1.clear()
    list_part2=[]
    list_part2.clear()
    for i in range(len(list1)//2):
        list_part1.append(list1[i])
        list_part2.append(list1[i+(len(list1)//2)])
        
    print("message_part1 : {}".format(list_part1))
    print("message_part2 : {}".format(list_part2))
        
    XOR_list=[]
    XOR_list.clear()
    for j in range(len(list1)//2):
        if list_part1[j]==list_part2[j]:
            XOR_list.append(0)
        else:
            XOR_list.append(1)
    
    print("XOR_message : {}".format(XOR_list))
    list2.extend(XOR_list)    
        
        

XOR_m=[]  
XOR_m.clear()

XOR_message(message,XOR_m)
print("XOR_m : {}".format(XOR_m))

message_part1 : [1, 1, 1, 0]
message_part2 : [1, 0, 0, 0]
XOR_message : [0, 1, 1, 0]
XOR_m : [0, 1, 1, 0]


In [4]:
m_XOR_m=[]  
m_XOR_m.clear()
m_XOR_m=message+XOR_m
print("m_XOR_m : {}".format(m_XOR_m))

m_XOR_m : [1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0]


In [5]:
preshared_string=[]  
preshared_string.clear()

def random_012str(list1,length):
    for i in range(length):
        list1.append(random.randint(0,2))
        
random_012str(preshared_string,(n*3)//4)
print("preshared_string : {}".format(preshared_string))

preshared_string : [2, 1, 1, 0, 0, 2]


In [6]:
# Step1 : depends on m||XOR(m), operate N |0> qubits with H,S(0) and H,Sdg(1)
print("m_XOR_m : {}".format(m_XOR_m))

# generate one qubit

q = QuantumRegister(1,'q')
c = ClassicalRegister(2,'c')

# Alice
#m_circ0 = QuantumCircuit(q,c)  # |+y>
#m_circ0.h(q[0])  
#m_circ0.s(q[0]) 
#m_circ0.barrier(q)

#m_circ1 = QuantumCircuit(q,c)  # |-y>
#m_circ1.h(q[0])
#m_circ1.sdg(q[0])
#m_circ1.barrier(q)

m_circ0 = QuantumCircuit(q,c)  # |+>
m_circ0.h(q[0])  
#m_circ0.barrier(q)

m_circ1 = QuantumCircuit(q,c)  # |->
m_circ1.x(q[0])
m_circ1.h(q[0])
#m_circ1.barrier(q)


#m_circ0.draw()
m_circ1.draw()

m_XOR_m : [1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0]


In [7]:
# generate circuits

m_circ=[m_circ0,m_circ1]

circuits_m_XOR_m=[]
circuits_m_XOR_m.clear()

for i in range((n*3)//2):
    circuits_m_XOR_m.append(m_circ[m_XOR_m[i]])
    
print("length of circuits_m_XOR_m : {}".format(len(circuits_m_XOR_m)))
circuits_m_XOR_m[0].draw()

length of circuits_m_XOR_m : 12


In [8]:
# Step2 : depends on Pre-shared String, insert decoy photons (|+> or |->) for each two m_XOR_m qubits
#         0 --> insert in the middle, 1 --> insert at the left side, 2 --> insert at the right side
print("preshared_string : {}".format(preshared_string))

decoy=[]
decoy.clear()
random_01str(decoy,(n*3)//4)
print("decoy ,0(|+>) or 1(|->) : {}".format(decoy))


Decoy_circ2 = QuantumCircuit(q,c)  # |+>
Decoy_circ2.h(q[0])  
#Decoy_circ2.barrier(q)

Decoy_circ3 = QuantumCircuit(q,c)  # |->
Decoy_circ3.x(q[0])
Decoy_circ3.h(q[0])
#Decoy_circ3.barrier(q)


#Decoy_circ2.draw()
Decoy_circ3.draw()

preshared_string : [2, 1, 1, 0, 0, 2]
decoy ,0(|+>) or 1(|->) : [1, 0, 0, 0, 1, 1]


In [9]:
# generate circuits

Decoy_circ=[Decoy_circ2,Decoy_circ3]

circuits_Decoy=[]
circuits_Decoy.clear()

for i in range((n*3)//4):
    circuits_Decoy.append(Decoy_circ[decoy[i]])
    
print("length of circuits_Decoy : {}".format(len(circuits_Decoy)))
circuits_Decoy[0].draw()

length of circuits_Decoy : 6


In [10]:
circuits_MD_step2=[]
circuits_MD_step2.clear()

for i in range((n*3)//4):
    if preshared_string[i]==0:  # mdm
        circuits_MD_step2.append(circuits_m_XOR_m[2*i])
        circuits_MD_step2.append(circuits_Decoy[i])
        circuits_MD_step2.append(circuits_m_XOR_m[2*i+1])
    elif preshared_string[i]==1:  #dmm
        circuits_MD_step2.append(circuits_Decoy[i])
        circuits_MD_step2.append(circuits_m_XOR_m[2*i])
        circuits_MD_step2.append(circuits_m_XOR_m[2*i+1])
    elif preshared_string[i]==2:  #mmd
        circuits_MD_step2.append(circuits_m_XOR_m[2*i])
        circuits_MD_step2.append(circuits_m_XOR_m[2*i+1])
        circuits_MD_step2.append(circuits_Decoy[i])
    

    
print("length of circuits_MD_step2 : {}".format(len(circuits_MD_step2)))
circuits_MD_step2[0].draw()

length of circuits_MD_step2 : 18


In [11]:
# Step3 : Alice_lock                          

# M -> X,Y,Z 

#Alice_lock_X=[]  
#Alice_lock_X.clear()    
#random_01str(Alice_lock_X,((n*3)//2))

#Alice_lock_Y=[]  
#Alice_lock_Y.clear()    
#random_01str(Alice_lock_Y,((n*3)//2))

#Alice_lock_Z=[]  
#Alice_lock_Z.clear()    
#random_01str(Alice_lock_Z,((n*3)//2))

Alice_lock_XYZ=[]  
Alice_lock_XYZ.clear()    
random_012str(Alice_lock_XYZ,((n*3)//2))

#print("Alice_lock_X : {}".format(Alice_lock_X))
#print("length of Alice_lock_X : {}".format(len(Alice_lock_X)))
#print("Alice_lock_Y : {}".format(Alice_lock_Y))
#print("length of Alice_lock_Y : {}".format(len(Alice_lock_Y)))
#print("Alice_lock_Z : {}".format(Alice_lock_Z))
#print("length of Alice_lock_Z : {}".format(len(Alice_lock_Z)))

print("Alice_lock_XYZ : {}".format(Alice_lock_XYZ))
print("length of Alice_lock_XYZ : {}".format(len(Alice_lock_XYZ)))


# D -> Rx,Ry,Rz

def random_lock(list1,length):
    for i in range(length):
        list1.append(random.randint(0,360))
        

Alice_lock_rx=[]  
Alice_lock_rx.clear()    
random_lock(Alice_lock_rx,((n*3)//4))

Alice_lock_ry=[]  
Alice_lock_ry.clear()    
random_lock(Alice_lock_ry,((n*3)//4))

Alice_lock_rz=[]  
Alice_lock_rz.clear()    
random_lock(Alice_lock_rz,((n*3)//4))

print("Alice_lock_rx : {}".format(Alice_lock_rx))
#print("length of Alice_lock_rx : {}".format(len(Alice_lock_rx)))
print("Alice_lock_ry : {}".format(Alice_lock_ry))
#print("length of Alice_lock_ry : {}".format(len(Alice_lock_ry)))
print("Alice_lock_rz : {}".format(Alice_lock_rz))
#print("length of Alice_lock_rz : {}".format(len(Alice_lock_rz)))

Alice_lock_XYZ : [1, 2, 0, 0, 2, 2, 1, 1, 0, 2, 1, 2]
length of Alice_lock_XYZ : 12
Alice_lock_rx : [289, 116, 246, 222, 150, 80]
Alice_lock_ry : [280, 154, 56, 21, 344, 128]
Alice_lock_rz : [44, 86, 242, 285, 326, 28]


In [12]:
circuits_Alice_lock_M=[]
circuits_Alice_lock_M.clear()

for i in range(((n*3)//2)):
    lock_circ = QuantumCircuit(q,c)
    #if Alice_lock_X[i]==1:
    #    lock_circ.x(q[0])
    #if Alice_lock_Y[i]==1:
    #    lock_circ.y(q[0])
    #if Alice_lock_Z[i]==1:
    #    lock_circ.z(q[0])
    
    if Alice_lock_XYZ[i]==0:
        lock_circ.x(q[0])
    if Alice_lock_XYZ[i]==1:
        lock_circ.y(q[0])
    if Alice_lock_XYZ[i]==2:
        lock_circ.z(q[0])
        
    circuits_Alice_lock_M.append(lock_circ)
    
print("length of circuits_Alice_lock_M : {}".format(len(circuits_Alice_lock_M)))
circuits_Alice_lock_M[0].draw()

length of circuits_Alice_lock_M : 12


In [13]:
circuits_Alice_lock_D=[]
circuits_Alice_lock_D.clear()

for i in range(((n*3)//4)):
    lock_circ = QuantumCircuit(q,c) 
    lock_circ.rx(2*np.pi*Alice_lock_rx[i]/360,q[0])
    lock_circ.ry(2*np.pi*Alice_lock_ry[i]/360,q[0])
    lock_circ.rz(2*np.pi*Alice_lock_rz[i]/360,q[0])
    #lock_circ.barrier(q)
    
    circuits_Alice_lock_D.append(lock_circ)
    
print("length of circuits_Alice_lock_D : {}".format(len(circuits_Alice_lock_D)))
circuits_Alice_lock_D[0].draw()

length of circuits_Alice_lock_D : 6


In [14]:
circuits_MD_step3=[]
circuits_MD_step3.clear()

for i in range((n*3)//4):
    if preshared_string[i]==0:  # mdm
        circuits_MD_step3.append(circuits_MD_step2[3*i]+circuits_Alice_lock_M[2*i])
        circuits_MD_step3.append(circuits_MD_step2[3*i+1]+circuits_Alice_lock_D[i])
        circuits_MD_step3.append(circuits_MD_step2[3*i+2]+circuits_Alice_lock_M[2*i+1])
    elif preshared_string[i]==1:  #dmm
        circuits_MD_step3.append(circuits_MD_step2[3*i]+circuits_Alice_lock_D[i])
        circuits_MD_step3.append(circuits_MD_step2[3*i+1]+circuits_Alice_lock_M[2*i])
        circuits_MD_step3.append(circuits_MD_step2[3*i+2]+circuits_Alice_lock_M[2*i+1])
    elif preshared_string[i]==2:  #mmd
        circuits_MD_step3.append(circuits_MD_step2[3*i]+circuits_Alice_lock_M[2*i])
        circuits_MD_step3.append(circuits_MD_step2[3*i+1]+circuits_Alice_lock_M[2*i+1])
        circuits_MD_step3.append(circuits_MD_step2[3*i+2]+circuits_Alice_lock_D[i])

    

print("length of circuits_MD_step3 : {}".format(len(circuits_MD_step3)))
#circuits_MD_step3[0].draw()
circuits_MD_step3[1].draw()

length of circuits_MD_step3 : 18


In [15]:
# Step4 : Send to Bob and Bob operates M with Bob_lock -> U3

def random_lock_U3(list1,length):
    for i in range(length):
        list1.append(random.randint(-4,4))

Bob_lock_U3x=[]  
Bob_lock_U3x.clear()    
random_lock_U3(Bob_lock_U3x,((n*3)//2))

Bob_lock_U3y=[]  
Bob_lock_U3y.clear()    
random_lock_U3(Bob_lock_U3y,((n*3)//2))

Bob_lock_U3z=[]  
Bob_lock_U3z.clear()    
random_lock_U3(Bob_lock_U3z,((n*3)//2))

print("Bob_lock_U3x : {}".format(Bob_lock_U3x))
#print("length of Bob_lock_U3x : {}".format(len(Bob_lock_U3x)))
print("Bob_lock_U3y : {}".format(Bob_lock_U3y))
#print("length of Bob_lock_U3y : {}".format(len(Bob_lock_U3y)))
print("Bob_lock_U3z : {}".format(Bob_lock_U3z))
#print("length of Bob_lock_U3z : {}".format(len(Bob_lock_U3z)))

Bob_lock_U3x : [-4, -3, 1, 3, -2, -1, -4, 4, -4, 4, -4, -2]
Bob_lock_U3y : [3, -3, 3, -4, 4, -1, 0, -1, -2, 0, 3, -1]
Bob_lock_U3z : [3, -2, -2, 1, -1, -4, 4, 4, 2, 4, 0, 4]


In [16]:
circuits_Bob_lock=[]
circuits_Bob_lock.clear()

for i in range(((n*3)//2)):
    lock_circ = QuantumCircuit(q,c) 
    lock_circ.u3((Bob_lock_U3x[i]/2)*np.pi,(Bob_lock_U3y[i]/2)*np.pi,(Bob_lock_U3z[i]/2)*np.pi,q[0])
    #lock_circ.barrier(q)
    circuits_Bob_lock.append(lock_circ)
    
print("length of circuits_Bob_lock : {}".format(len(circuits_Bob_lock)))
circuits_Bob_lock[0].draw()

length of circuits_Bob_lock : 12


In [17]:
circuits_MD_step4=[]
circuits_MD_step4.clear()

for i in range((n*3)//4):
    if preshared_string[i]==0:  # mdm
        circuits_MD_step4.append(circuits_MD_step3[3*i]+circuits_Bob_lock[2*i])
        circuits_MD_step4.append(circuits_MD_step3[3*i+1])
        circuits_MD_step4.append(circuits_MD_step3[3*i+2]+circuits_Bob_lock[2*i+1])
    elif preshared_string[i]==1:  #dmm
        circuits_MD_step4.append(circuits_MD_step3[3*i])
        circuits_MD_step4.append(circuits_MD_step3[3*i+1]+circuits_Bob_lock[2*i])
        circuits_MD_step4.append(circuits_MD_step3[3*i+2]+circuits_Bob_lock[2*i+1])
    elif preshared_string[i]==2:  #mmd
        circuits_MD_step4.append(circuits_MD_step3[3*i]+circuits_Bob_lock[2*i])
        circuits_MD_step4.append(circuits_MD_step3[3*i+1]+circuits_Bob_lock[2*i+1])
        circuits_MD_step4.append(circuits_MD_step3[3*i+2])
    

print("length of circuits_MD_step4 : {}".format(len(circuits_MD_step4)))
#circuits_MD_step4[0].draw()
circuits_MD_step4[1].draw()

length of circuits_MD_step4 : 18


In [18]:
# Step5 : Send to Alice, and Alice checks decoy

circuits_Alice_check_decoy=[]
circuits_Alice_check_decoy.clear()

circuits_Alice_m_after_check=[]
circuits_Alice_m_after_check.clear()

for i in range((n*3)//4):
    if preshared_string[i]==0:  # mdm
        circuits_Alice_m_after_check.append(circuits_MD_step4[3*i])
        circuits_Alice_check_decoy.append(circuits_MD_step4[3*i+1])
        circuits_Alice_m_after_check.append(circuits_MD_step4[3*i+2])
    elif preshared_string[i]==1:  #dmm
        circuits_Alice_check_decoy.append(circuits_MD_step4[3*i])
        circuits_Alice_m_after_check.append(circuits_MD_step4[3*i+1])
        circuits_Alice_m_after_check.append(circuits_MD_step4[3*i+2])
    elif preshared_string[i]==2:  #mmd
        circuits_Alice_m_after_check.append(circuits_MD_step4[3*i])
        circuits_Alice_m_after_check.append(circuits_MD_step4[3*i+1])
        circuits_Alice_check_decoy.append(circuits_MD_step4[3*i+2])
    

    
print("length of circuits_Alice_check_decoy : {}".format(len(circuits_Alice_check_decoy)))
print("length of circuits_Alice_m_after_check : {}".format(len(circuits_Alice_m_after_check)))

circuits_Alice_check_decoy[0].draw()
#circuits_Alice_m_after_check[0].draw()

length of circuits_Alice_check_decoy : 6
length of circuits_Alice_m_after_check : 12


In [19]:
# decoy unlock

circuits_Alice_unlock_D=[]
circuits_Alice_unlock_D.clear()

for i in range(((n*3)//4)):
    unlock_circ = QuantumCircuit(q,c) 
    unlock_circ.rz(2*np.pi*(360-Alice_lock_rz[i])/360,q[0])
    unlock_circ.ry(2*np.pi*(360-Alice_lock_ry[i])/360,q[0])
    unlock_circ.rx(2*np.pi*(360-Alice_lock_rx[i])/360,q[0])
    #unlock_circ.barrier(q)
    
    circuits_Alice_unlock_D.append(unlock_circ)
    
print("length of circuits_Alice_unlock_D : {}".format(len(circuits_Alice_unlock_D)))
circuits_Alice_unlock_D[0].draw()

length of circuits_Alice_unlock_D : 6


In [20]:
# X measurement

Alice_X_measurement = QuantumCircuit(q,c) # X basis
Alice_X_measurement.barrier(q)
Alice_X_measurement.h(q[0])
Alice_X_measurement.measure(q[0],c[0])

In [21]:
circuits_Alice_unlock_and_measure_decoy=[]
circuits_Alice_unlock_and_measure_decoy.clear()

for i in range(((n*3)//4)):
    circuits_Alice_unlock_and_measure_decoy.append(circuits_Alice_check_decoy[i]+circuits_Alice_unlock_D[i]+Alice_X_measurement)
    

print("length of circuits_Alice_unlock_and_measure_decoy : {}".format(len(circuits_Alice_unlock_and_measure_decoy)))

circuits_Alice_unlock_and_measure_decoy[0].draw()

length of circuits_Alice_unlock_and_measure_decoy : 6


In [22]:
from qiskit import execute, BasicAer
backend_sim = BasicAer.get_backend('qasm_simulator')

job_sim1 = qiskit.execute(circuits_Alice_unlock_and_measure_decoy, backend_sim, shots=1024)
result_sim1 = job_sim1.result()
counts1 = result_sim1.get_counts(circuits_Alice_unlock_and_measure_decoy[0])
print(counts1)

{'01': 1024}


In [23]:
# get result of Alice measuring decoy photons

Alice_measure_decoy_result=[]
Alice_measure_decoy_result.clear()

for i in range(((n*3)//4)):
    Alice_measure_decoy_result.append(int((list(result_sim1.get_counts(circuits_Alice_unlock_and_measure_decoy[i]))[0])[1]))
    
print("Alice_measure_decoy_result = {}".format(Alice_measure_decoy_result))

Alice_measure_decoy_result = [1, 0, 0, 0, 1, 1]


In [24]:
# Check result of decoy photons
right_times_decoy = 0

for i in range(((n*3)//4)):
    if decoy[i]==Alice_measure_decoy_result[i]:
        right_times_decoy+=1
        print("measure {} = origin {}".format(Alice_measure_decoy_result[i],decoy[i]))
    else :
        print("measure {} != origin {}".format(Alice_measure_decoy_result[i],decoy[i]))
        print("Different result has been found.")


print("")
        
error_rate_decoy=((((n*3)//4)-right_times_decoy)/((n*3)//4))
print("error rate = {}%".format(error_rate_decoy*100))

measure 1 = origin 1
measure 0 = origin 0
measure 0 = origin 0
measure 0 = origin 0
measure 1 = origin 1
measure 1 = origin 1

error rate = 0.0%


In [25]:
# Step6 : If no Eve, Alice recovers m 

circuits_Alice_unlock_m_after_check=[]
circuits_Alice_unlock_m_after_check.clear()

for i in range(((n*3)//2)):
    circuits_Alice_unlock_m_after_check.append(circuits_Alice_m_after_check[i] + circuits_Alice_lock_M[i])
    

print("length of circuits_Alice_unlock_m_after_check : {}".format(len(circuits_Alice_unlock_m_after_check)))

circuits_Alice_unlock_m_after_check[0].draw()

length of circuits_Alice_unlock_m_after_check : 12


In [26]:
# Step7 : Send to Bob, and Bob unlocks M

circuits_Bob_unlock_m=[]
circuits_Bob_unlock_m.clear()

for i in range(((n*3)//2)):
    unlock_circ = QuantumCircuit(q,c) 
    unlock_circ.u3(((4-Bob_lock_U3x[i])/2)*np.pi,((4-Bob_lock_U3y[i])/2)*np.pi,((4-Bob_lock_U3z[i])/2)*np.pi,q[0])
    #unlock_circ.barrier(q)
    circuits_Bob_unlock_m.append(unlock_circ)
    
print("length of circuits_Bob_unlock_m : {}".format(len(circuits_Bob_unlock_m)))
circuits_Bob_unlock_m[0].draw()

length of circuits_Bob_unlock_m : 12


In [27]:
# Y measurement

Bob_Y_measurement = QuantumCircuit(q,c) # Y basis
Bob_Y_measurement.barrier(q)
#Bob_Y_measurement.rx((1/2)*np.pi,q[0])
Bob_Y_measurement.h(q[0])
#Bob_Y_measurement.s(q[0])
Bob_Y_measurement.measure(q[0],c[0])

Bob_Y_measurement.draw()

In [28]:
circuits_Bob_unlock_m_and_measure=[]
circuits_Bob_unlock_m_and_measure.clear()

for i in range(((n*3)//2)):
    circuits_Bob_unlock_m_and_measure.append(circuits_Alice_unlock_m_after_check[i]+circuits_Bob_unlock_m[i]+Bob_Y_measurement)
    
print("length of circuits_Bob_unlock_m_and_measure : {}".format(len(circuits_Bob_unlock_m_and_measure)))
circuits_Bob_unlock_m_and_measure[0].draw()

length of circuits_Bob_unlock_m_and_measure : 12


In [29]:
job_sim2 = qiskit.execute(circuits_Bob_unlock_m_and_measure, backend_sim, shots=1024)
result_sim2 = job_sim2.result()
counts2 = result_sim2.get_counts(circuits_Bob_unlock_m_and_measure[0])
print(counts2)

{'01': 1024}


In [30]:
# get result of Bob measuring M photons

Bob_measure_M_result=[]
Bob_measure_M_result.clear()

for i in range(((n*3)//2)):
    Bob_measure_M_result.append(int((list(result_sim2.get_counts(circuits_Bob_unlock_m_and_measure[i]))[0])[1]))
    
print("Bob_measure_M_result = {}".format(Bob_measure_M_result))

Bob_measure_M_result = [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1]


In [ ]:
1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0]